<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background- padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      4D Analytics using the New York City Taxi dataset --Temporal and Timeseries
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>
This is a demonstration of Vantage capabilities for
    <ul style = 'font-size:16px;font-family:Arial'>
<li>Temporal using the PERIOD data type, CONTAINS, OVERLAPS and EXPAND</li>
        <li>Timeseries and Primary Time Index</li>
        </ul>
    </p>
    

<hr style='height:2px;border:none;'>
<p style = 'font-size:28px;font-family:Arial'><b>1. Connect to Vantage</b></p>

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell.</p>

In [ ]:
%connect local, hidewarnings=true

<p style = 'font-size:16px;font-family:Arial'>Setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>

In [ ]:
SET query_band='DEMO=NYC_Taxi_Temporal_SQL.ipynb;' UPDATE FOR SESSION;

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'> <b>1.1 Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. In this demo since we are using Temporal table we will be creating databases and tables in local storage and use them in the notebook. Please execute the procedure in the next cell.</p>

In [ ]:
-- call get_data('DEMO_NYCTaxi_cloud');    -- Takes about 2 minutes
call get_data('DEMO_NYCTaxi_local');    -- takes about 3 minutes

<p style = 'font-size:16px;font-family:Arial'>Optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
call space_report();  -- optional, takes about 10 seconds

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'> <b>1.2 Access data in Vantage  </b> </p>
<p style = 'font-size:16px;font-family:Arial'>Let us check the data sample. This demonstration will use two tables: the taxi trip details and the fares for each trip. The queries below will sample each table and then show the range of the time period covered by the data. </p>

In [ ]:
SELECT TOP 5 * from DEMO_NYCTaxi.Trip;

In [ ]:
SELECT TOP 5 * from DEMO_NYCTaxi.Trip_Fare;

In [ ]:
SELECT MIN(pickup_datetime), MAX(dropoff_datetime) from DEMO_NYCTaxi.Trip;

<hr style='height:2px;border:none;'>
<p style = 'font-size:28px;font-family:Arial'><b>2. Warmup: temporal algebra </b></p>
<img  src="contains.png" alt="Contains Example" width="600" align="right" />
<p style = 'font-size:16px;font-family:Arial'> Let's warmup with a bit of temporal algebra and get familiar with period types and operators. 
<br><br>
For a taxi service, the number of dispatchers needed is impacted by the number of pickups. While some pickups are based on a taxi stand or someone waving their hand along the side of the street, the driver calls in the trip. 
<br><br>The period data type may be based on dates or timestamps and has a beginning bound and extends up to, but does not include the ending bound value. The ending bound may be expressed as "UNTIL_CHANGED" or "UNTIL CLOSED" which effectively means "forever". Most commonly, the period data type would appear in a table but in the example below, we will use a literal to count the number of taxi pickups between 10:30 and 10:45 on Nov 10th.</p>

In [ ]:
SELECT COUNT(1)
FROM
    DEMO_NYCTaxi.Trip
-- WHERE pickup_datetime BETWEEN '2013-11-10 10:30:00' AND '2013-11-10 10:45:00'
-- Here is an alternative to the above using the period data type
WHERE
    PERIOD('2013-11-10 10:30:00' (TIMESTAMP), '2013-11-10 10:45:00' (TIMESTAMP)) CONTAINS pickup_datetime;

<img  src="contains-period.png" alt="Contains Period Example" width="600" align="right" />
<p style = 'font-size:16px;font-family:Arial'> 
The above example used CONTAINS for a simple comparison to a date which is easily replaced by BETWEEN.  A more interesting case is when two periods are in a constraint which would be more complicated logic.  The following query analyzes how many rides started and ended between 10:30 and 10:45 on Nov 10th.

In [ ]:
SELECT COUNT(1)
FROM
    DEMO_NYCTaxi.Trip
WHERE
    pickup_datetime < dropoff_datetime
AND
    PERIOD('2013-11-10 10:30:00' (TIMESTAMP), '2013-11-10 10:45:00' (TIMESTAMP)) CONTAINS PERIOD(pickup_datetime, dropoff_datetime);

<img  src="overlap.png" alt="Overlap Example" width="600" align="right" />
<p style = 'font-size:16px;font-family:Arial'> 
The taxi cabs now contain video screens that include advertising content. To analyze the number of people that may be viewing the video to justify compensation for advertising, we need to know how many people are in the cabs in a given period. 
<br><br>
Where the CONTAINS operator above says that the timestamp or period must be within the bounds of the period, the OVERLAPS operator selects any rows where the row's period begins, ends, or overlaps the constraint period.  In the following example, we count how many passengers were in a taxi between 10:30 and 10:45 on Nov 10th.

In [ ]:
SELECT SUM(passenger_count)
FROM
    DEMO_NYCTaxi.Trip
WHERE
    pickup_datetime < dropoff_datetime
    AND
    PERIOD('2013-11-10 10:30:00' (TIMESTAMP), '2013-11-10 10:45:00' (TIMESTAMP))
        OVERLAPS PERIOD(pickup_datetime, dropoff_datetime);

<img  src="normalize.png" alt="Overlap Example" width="600" align="right" />
<p style = 'font-size:16px;font-family:Arial'> 
As input to the hours of operations, management would like to know how many hours in the day there is at least 1 taxi operating.
<br><br>
Let's assume we run the fleet of the 3 taxis identifiable by medallion starting with ‘007’. The fleet is considered active if at least 1 taxi is driving. For how long the fleet has been “active” on November 10th?
<br><br>
We will need to consolidate (or normalize) the periods where taxis are active and then find the duration of the resulting periods of activity. The final query will have nested derived tables, so we will start by showing a sample of the innermost query which will create a set of period on the date for the group of medallions:

In [ ]:
SELECT TOP 5
    PERIOD(pickup_datetime, dropoff_datetime) AS norm_per
FROM 
    DEMO_NYCTaxi.Trip
WHERE 
    CAST(pickup_datetime AS DATE) = DATE '2013-11-10'
    AND medallion LIKE '007%'
ORDER BY 1;

<p style = 'font-size:16px;font-family:Arial'> We will then use the NORMALIZE modifier to consolidate the periods. With NORMALIZE, the result of the select is normalized on the first period column in the select list. Period values that meet or overlap are coalesced, that is, combined to form a period that
encompasses the individual period values.  For comparison, we will follow with a query that doesn't normalize which is the total hours of active service of all of the taxis. 

In [ ]:
SELECT SUM(INTERVAL(normalized_periods) HOUR(2) TO MINUTE) AS normalized_activity
FROM (
    SELECT NORMALIZE PERIOD(pickup_datetime, dropoff_datetime) AS normalized_periods
    FROM
        DEMO_NYCTaxi.Trip
    WHERE
        CAST(pickup_datetime AS DATE) = DATE '2013-11-10'
        AND medallion LIKE '007%'
) d1
ORDER BY 1;

In [ ]:
SELECT SUM(INTERVAL(PERIOD(pickup_datetime, dropoff_datetime)) HOUR TO MINUTE) AS total_activity
FROM
    DEMO_NYCTaxi.Trip
WHERE
    CAST(pickup_datetime AS DATE) = DATE '2013-11-10'
    AND medallion LIKE '007%';

<p style = 'font-size:16px;font-family:Arial'> The normalized activity consolidating all of the periods before summarizing is less than the total activity. 
<br><br>
There are situations where you need to see the state of the business at fixed intervals. For example: total inventory at the end of each week.  The EXPAND ON operator will create those routine periods for reporting. To demonstrate EXPAND ON, we will get a count of active cabs at the top of each hour.

In [ ]:
SELECT begin_hour, COUNT(*)
FROM (
    SELECT begin(hour_check) AS begin_hour
    FROM DEMO_NYCTaxi.Trip  
    WHERE (pickup_datetime (DATE)) = '2013-11-10'
        AND medallion LIKE '007%'
    EXPAND ON PERIOD(pickup_datetime, dropoff_datetime) AS hour_check BY ANCHOR ANCHOR_HOUR
) a
GROUP BY 1
ORDER BY 1;

In [ ]:
%chart x = begin_hour, y = count(*), title = "Count of active cars of Medallian 007 by hour", height = 300, width = 400

<hr style='height:2px;border:none;'>
<p style = 'font-size:28px;font-family:Arial'><b>3. Timeseries Analysis </b></p>
<p style = 'font-size:16px;font-family:Arial'> Now we have seen the trip and fare details, Let's do some analysis based on time.
The query below issues a GROUP BY clause referencing a unit of time and includes a USING TIMECODE clause to specify a column that the unit grouping is to be applied to. <br><br>
<p style = 'font-size:16px;font-family:Arial'>    
How many passengers are being picked up by hour?</p>

In [ ]:
SELECT TOP 5
    EXTRACT(DAY FROM pickup_datetime),
    EXTRACT(HOUR FROM pickup_datetime),
    SUM(passenger_count)
FROM
    DEMO_NYCTaxi.Trip
GROUP BY 1, 2
ORDER BY 1, 2;

In [ ]:
SELECT TOP 72
       $TD_TIMECODE_RANGE,
       BEGIN($TD_TIMECODE_RANGE) AS time_bucket_start,    -- (timestamp, format 'YYYY-MM-DDBHH:MI:SS') time_bucket_start
       SUM(passenger_count) AS passenger_count
FROM
    DEMO_NYCTaxi.Trip
GROUP BY
    TIME(HOURS(1))
USING
    TIMECODE(pickup_datetime)
ORDER BY 1;

<p style = 'font-size:16px;font-family:Arial'>It's about time to add some visual element...

In [ ]:
%chart x = time_bucket_start, y = passenger_count, title = "Passenger pickup by hour"

<p style = 'font-size:16px;font-family:Arial'>How many passengers are being picked up and what is the average trip duration by vendor every 15min?

In [ ]:
SELECT TOP 96
       $TD_TIMECODE_RANGE,
       vendor_id,
       SUM(passenger_count),
       AVG(trip_time_in_secs)
FROM
    DEMO_NYCTaxi.Trip
GROUP BY
    TIME(MINUTES(15) AND vendor_id)
USING
    TIMECODE(pickup_datetime)
ORDER BY 1,2;

<p style = 'font-size:16px;font-family:Arial'> Let's call this a view

In [ ]:
REPLACE VIEW NYC_taxi_trip_ts AS
SELECT
  $TD_TIMECODE_RANGE time_bucket_per_15min,
  vendor_id,
  SUM(passenger_count) passenger_cnt,
  SUM(trip_time_in_secs) avg_trip_time_in_secs
FROM
    DEMO_NYCTaxi.Trip 
GROUP BY
    TIME(MINUTES(15) AND vendor_id)
USING
    TIMECODE(pickup_datetime);

<p style = 'font-size:16px;font-family:Arial'><b>Moving Averages</b>
    <br>
 Let's calculate a 2 hours minutes moving average on our 15-minutes time series. 2 hour is 8 * 15 minutes periods.

In [ ]:
SELECT TOP 5 * FROM MovingAverage (
    ON NYC_taxi_trip_ts
    PARTITION BY vendor_id ORDER BY time_bucket_per_15min
    USING
        MAvgType ('S')
        WindowSize (8)
        TargetColumns ('passenger_cnt')
) AS dt 
WHERE
    BEGIN(time_bucket_per_15min)(DATE) = '2013-11-10'
ORDER BY
    vendor_id,
    time_bucket_per_15min;

<p style = 'font-size:16px;font-family:Arial'> We can use this to identify demand trends for each time period. A basic way to do this would be to check if the current pickup count is above (rising) or below (reducing) our moving average.

In [ ]:
SELECT
    time_bucket_per_15min,
    vendor_id,
    passenger_cnt,
    CASE WHEN passenger_cnt - passenger_cnt_smavg > 0 THEN '+' ELSE '-' END trend
FROM MovingAverage (
    ON NYC_taxi_trip_ts
    PARTITION BY vendor_id
    ORDER BY time_bucket_per_15min
    USING
        MAvgType ('S')
        WindowSize (8)
        TargetColumns ('passenger_cnt')
) AS dt 
WHERE
    BEGIN(time_bucket_per_15min)(DATE) = '2013-11-10'
ORDER BY
    vendor_id,
    time_bucket_per_15min;

In [ ]:
%chart x = time_bucket_per_15min, y = passenger_cnt, groupingcol = vendor_id,
mark = line, title = "passenger count", height = 300, width = 1000

<p style = 'font-size:16px;font-family:Arial'> We can use this to compare how vendors are doing.

In [ ]:
SELECT TOP 5 dt.*
FROM (
    SELECT
        time_bucket_per_15min,
        vendor_id,
        passenger_cnt,
        CASE WHEN passenger_cnt - passenger_cnt_smavg > 0 THEN '+' ELSE '-' END trend
    FROM MovingAverage (
    ON NYC_taxi_trip_ts
    PARTITION BY vendor_id
    ORDER BY time_bucket_per_15min
    USING
        MAvgType ('S')
        WindowSize (8)
        TargetColumns ('passenger_cnt')
    ) AS dt 
    WHERE
        BEGIN(time_bucket_per_15min)(DATE) = '2013-11-10'
) dt
PIVOT (
  MAX(passenger_cnt) AS passenger_cnt,
  MAX(trend) AS trend FOR vendor_id IN ('CMT', 'VTS')
) dt
ORDER BY 1;

<hr style='height:2px;border:none;'>
<p style = 'font-size:28px;font-family:Arial'><b>4.  Clean up </b></p>
<p style = 'font-size:18px;font-family:Arial'> <b>4.1 Database and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
call remove_data('DEMO_NYCTaxi');    -- Takes about 10 seconds, optional if you want to use the data later
-- Note that the same database & tables are used in Usecases/NYC-taxi-4d/NYC-taxi-geospatial.ipynb 

<hr style='height:2px;border:none;'>
<p style = 'font-size:28px;font-family:Arial'><b>5. Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial'>
In this demonstration we have seen that Temporal and period datatype is time aware and the queries using the same are fairly simple and easy to use. We have also seen how we can create a time index and use it in queries. 

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>5.2 Reference Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
        <li>Introduction to Teradata Time Series Tables and Operations: <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Time-Series-Tables-and-Operations-17.20'>https://docs.teradata.com/r/Teradata-VantageTM-Time-Series-Tables-and-Operations-17.20</a></li>
</ul>

<footer style="padding-bottom:35px; background:#91A0AB; ">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>